# Getting Started with Azure AI Search for Vector Database

In [ ]:
! pip install azure-identity
! pip install openai
! pip install python-dotenv
! pip install azure-search-documents

In [4]:
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    VectorizedQuery,
 
)

# Set up OpenAI client based on environment variables
load_dotenv()
AZURE_OPENAI_ENDPOINT: str = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY: str = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION: str = "2023-05-15"
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME: str = os.getenv(
    "AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME"
)

# Set up Azure AI Search client based on environment variables
AZURE_SEARCH_SERVICE_ENDPOINT: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_ADMIN_KEY: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)

# Set this flag to True if you are using Azure Active Directory
use_aad_for_aoai = True

if use_aad_for_aoai:
    # Use Azure Active Directory (AAD) authentication
    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_ad_token_provider=token_provider,
    )
else:
    # Use API key authentication
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
    )

# Set this flag to True if you are using Azure Active Directory
use_aad_for_search = True

if use_aad_for_search:
    # Use Azure Active Directory (AAD) authentication
    credential = DefaultAzureCredential()
else:
    # Use API key authentication
    credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)


# Example function to generate document embedding
def generate_embedding(text: str):
    # Generate embeddings for the provided text using the specified model
    embeddings_response = client.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME, input=text
    )
    # Extract the embedding data from the response
    return embeddings_response.data[0].embedding

# Azure AI Search retrieval modes

## Vector search is not always optimal
For example, consider searches for exact strings.

In [5]:
AZURE_SEARCH_FULL_INDEX = "msaitour-nyc-nba-rules"

# Pure Vector Search
query = "Technical Foul 16"
  
search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=credential)
vector_query = VectorizedQuery(vector=generate_embedding(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
)  
  

for doc in results:
    found_content = "Not found"
    if query in doc['chunk']:
        found_content = doc['chunk'][doc['chunk'].find(query):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Score: 0.87044	Matching text: Not found
Score: 0.86960	Matching text: Not found
Score: 0.86877	Matching text: Not found


Compare to a text search for same query:

In [6]:
results = search_client.search(  
    search_text=query,  
    top=3
)  

for doc in results:
    found_content = "Not found"
    if query in doc["chunk"]:
        found_content = doc["chunk"][doc["chunk"].find(query) :].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Score: 5.08329	Matching text: Technical Foul 16:                      $5,000 fine plus one-game suspension Each Additional Technical Foul:   $5,000 fine Each Two Additional Technical Fouls
Score: 4.05056	Matching text: Not found
Score: 3.61260	Matching text: Not found


## Hybrid retrieval
Uses RRF to combine vector and text results.

In [7]:
results = search_client.search(search_text=query, top=3)

for doc in results:
    found_content = "Not found"
    if query in doc["chunk"]:
        found_content = doc["chunk"][doc["chunk"].find(query) :].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Score: 5.08329	Matching text: Technical Foul 16:                      $5,000 fine plus one-game suspension Each Additional Technical Foul:   $5,000 fine Each Two Additional Technical Fouls
Score: 4.05056	Matching text: Not found
Score: 3.61260	Matching text: Not found


Hybrid ranking is good at recalling the relvant results not always optimal ranking.

In [10]:
query = "What is the penalty for a player who deliberately hangs on the rim after a dunk?"

vector_query = VectorizedQuery(
    vector=generate_embedding(query), k_nearest_neighbors=5, fields="vector"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
)

for result in results:  
    print(f"Content: {result['chunk']}")  
    print(f"Score: {result['@search.score']}\n")  

Content: player is nearby and the offensive player has 
the ball.

PENALTY: Loss of ball. The ball is awarded to the opposing team on the sideline, near- 
est the spot of the violation but no nearer the baseline than the foul line extended.

Section XI—Entering Basket From Below
A player shall not be the last to touch a ball which rises above the rim level within the 

cylinder from below.
PENALTY: Loss of ball. The ball is awarded to the opposing team on the sideline at 

the free throw line extended.

Section XII—Illegal Assist in Scoring
a. A player may not assist himself in an attempt to score by using any part of the rim, 

net, backboard, or basket support to lift, hold, or raise himself.
b. A player may not assist a teammate to gain height while attempting to score.
PENALTY: Loss of ball. The ball is awarded to the opposing team on the sideline at 

the free throw line extended.

Section XIII—Traveling
a. A player who receives the ball while standing still may pivot, using eithe

## Hybrid + Semantic Reranking 🎉

In [11]:
query = "What is the penalty for a player who deliberately hangs on the rim after a dunk?"

vector_query = VectorizedQuery(
    vector=generate_embedding(query), k_nearest_neighbors=5, fields="vector"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    query_type="semantic",
    semantic_configuration_name="msaitour-nyc-nba-rules-semantic-configuration",
)

for result in results:  
    print(f"Content: {result['chunk']}")  
    print(f"Score: {result['@search.reranker_score']}\n")  

Content: to have the requisite number of players eligible and able to play in 
a game, the suspensions will be served over multiple games (e.g., with some players suspended 
for the first game following the altercation and other players suspended for the second game 
following the altercation), with the players serving their suspensions alphabetically, according 
to the first letters of their last names.  

d. A player, coach, or assistant coach, upon being notified by an official that he/she has been 
ejected from the game, must leave the playing area IMMEDIATELY and remain in the dressing 
room of his/her team during such suspension until completion of the game or leave the building. 
Violation of this rule shall call for an automatic fine. A fine not to exceed $100,000 and possible 
forfeiture of the game may be imposed for any violation of this rule.

e. Any player who in the opinion of the officials has deliberately hung on the basket 
ring shall be assessed a non-unsportsmanlike 

## (BONUS) Hybrid + Semantic Reranking w/captions and answers 🎉🎉🎉

In [17]:
query = "What is the penalty for a player who deliberately hangs on the rim after a dunk?"

vector_query = VectorizedQuery(
    vector=generate_embedding(query), k_nearest_neighbors=10, fields="vector"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    query_type="semantic",
    semantic_configuration_name="msaitour-nyc-nba-rules-semantic-configuration",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=5,
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    # print(f"Content: {result['chunk']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Reranker Score: 2.3965227603912354
Caption: A fine not to exceed<em> $100,000</em> and possible  forfeiture of the game may be imposed for any violation of this rule. e. Any player who in the opinion of the officials has deliberately hung on the basket  ring shall be assessed a non-unsportsmanlike technical foul and a fine of at least<em> $2,000.</em>

Reranker Score: 2.121840476989746
Caption: a technical foul shall be assessed for unsportsmanlike tactics such as: (1) disrespectfully addressing an<em> official</em> (2) physically contacting an<em> official</em> (3) overt actions indicating resentment to a call or no-call (4) use of profanity (5) a coach entering onto the court without permission of an<em> official</em> (6) a<em> deliberately-thrown</em> elbow or any …

Reranker Score: 2.1199257373809814
Caption: Touch the rim, net, or ball while the ball is in the net, preventing it from clearing  the basket. PENALTY: If the violation is at the opponent’s basket, the offended team is 